In [2]:
# import thu vien YOLO
from ultralytics import YOLO

#Load model
model = YOLO("../Model/Models_yolov10n_dataBienSoNhieuLoaiv4_datanoscale_anhmau1nhan/runs/detect/train/weights/best.pt")

In [3]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.79  Python-3.9.19 torch-2.4.0+cpu CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)
Setup complete  (8 CPUs, 7.8 GB RAM, 148.9/229.5 GB disk)


In [5]:
# phai bat buoc cai goi C/C++ development cua visual studio
# Kiem tra torch va CUDA co phu hop?
import torch
print(torch.cuda.is_available())  # Kiểm tra xem CUDA có khả dụng không
print(torch.cuda.device_count())   # Số lượng GPU có sẵn

print(torch.__version__)

False
0
2.4.0+cpu


In [6]:
#Kiem tra torch ket noi GPU
import torch
torch.zeros(1).cuda()

AssertionError: Torch not compiled with CUDA enabled

In [4]:
# Làm xám ảnh
import cv2

def convert_to_grayscale(image_path, max_width=800, max_height=800):
    # Đọc bức ảnh từ đường dẫn
    image = cv2.imread(image_path)
    
    # Kiểm tra nếu ảnh không tồn tại hoặc không thể đọc
    if image is None:
        print("Không thể đọc ảnh. Vui lòng kiểm tra đường dẫn.")
        return None
    
    # Chuyển ảnh sang dạng grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Lấy kích thước ban đầu của ảnh
    original_height, original_width = gray_image.shape[:2]

    # Tính toán tỷ lệ để resize ảnh sao cho không vượt quá kích thước tối đa
    scaling_factor = min(max_width / original_width, max_height / original_height, 1)

    # Resize ảnh nếu cần
    if scaling_factor < 1:
        new_width = int(original_width * scaling_factor)
        new_height = int(original_height * scaling_factor)
        gray_image = cv2.resize(gray_image, (new_width, new_height))
    
    # Trả về ảnh xám đã được resize
    return gray_image

# Ví dụ sử dụng
gray_img = convert_to_grayscale("../Predict_Data/1.jpg")
#gray_img = convert_to_grayscale("./predict_data/z3945403596223_a1e193a942d30367b0b0bee724439713.jpg")
# Hiển thị ảnh xám nếu có
if gray_img is not None:
    cv2.imshow("Gray Image", gray_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
######### Detect và lấy ra các detected frame để lưu lại thành hình ảnh
import cv2
import os
from ultralytics import YOLO
import numpy as np

#### Load the YOLOv8 model
#model = YOLO("./Models_4_yolov8n_datav5/runs/detect/train/weights/best.pt")
#model = YOLO("./Models_5_yolov10n/runs/detect/train/weights/best.pt")
#model = YOLO("./Models_6_yolov10n_datav7/runs/detect/train/weights/best.pt")
# model = YOLO("../ThamKhao_Project/Computervisionprojects-main/ANPR_YOLOv10/weights/best.pt")
#model = YOLO("./Models_yolov10n_dataBienSoNhieuLoaiv1/runs/detect/train/weights/best.pt")
#model = YOLO("./Models_yolov10n_dataBienSoNhieuLoaiv4_datanoscale_anhmau1nhan/runs/detect/train/weights/best.pt")

#### Open the video file
#video_path = "./predict_data/VID_20220519_144812.mp4"
#video_path = "./predict_data/VID_20230506_163430.mp4"
#video_path = "./predict_data/101.jpg"
#video_path = "./predict_data/Video xe máy - xe hơi chạy trên đường.mp4"
#video_path = "./predict_data/video_1.gif"
#video_path = "./predict_data/VID_20220519_144619.mp4"
#video_path = "./predict_data/bike_counter_10min.mp4"
# video_path = "./predict_data/VID_20230506_163430.mp4"
video_path = "../../image/xemay/thanh-nien-dung-tren-yen-xe-may-dang-chay.jpg"

#video_path = "./predict_data/xe_2.jpg"
cap = cv2.VideoCapture(video_path)

#### Tạo thư mục để lưu các frame nếu chưa có
output_folder = "./detected_frames"
os.makedirs(output_folder, exist_ok=True)

frame_count = 0
confidence_threshold = 0 # Mức độ tin cậy cho trước

#### Kiểm tra xem tệp là ảnh hay video
is_image = video_path.lower().endswith(('.jpg', '.jpeg', '.png'))

#### Định nghĩa màu sắc cho từng nhãn
colors = {
    "bienso_do": (0, 0, 255),       # Màu đỏ cho biển quân sự
    "bienso_vang": (0, 255, 255),       # Màu vàng đậm cho biển vàng
    "bienso_xanh": (255, 255, 128),     # Màu xanh nhạt cho biển xanh
    "bienso": (0, 255, 0)               # Màu xanh lá cho biển thường
}

#### Xử lý ảnh nếu là tệp ảnh
if is_image:
    # Đọc ảnh
    frame = cv2.imread(video_path)
    
    # Thực hiện inference YOLOv8
    results = model(frame)
    
    # Kiểm tra nếu có bất kỳ đối tượng nào được phát hiện
    if len(results[0].boxes) > 0:
        for box in results[0].boxes:
            confidence = box.conf[0]
            label = box.cls  # Chỉ số nhãn lớp
            label_name = model.names[int(label)]  # Tên nhãn

            # Kiểm tra nếu độ tin cậy trên ngưỡng
            if confidence > confidence_threshold:
                # Lấy tọa độ bounding box
                x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())

                # Cắt phần ảnh phát hiện
                detected_object = frame[y1:y2, x1:x2]

                # Lưu ảnh đã cắt kèm tên nhãn và độ tin cậy trong tên file
                frame_filename = os.path.join(output_folder, f"{label_name}_{confidence:.2f}_frame_{frame_count:04d}.jpg")
                cv2.imwrite(frame_filename, detected_object)

                # Xác định màu sắc dựa trên tên nhãn
                color = colors.get(label_name, (255, 255, 255))

                # Vẽ bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                
                # Hiển thị nhãn và độ tin cậy
                text = f"{label_name} {confidence:.2f}"
                cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

        # Hiển thị kết quả xử lý ảnh
        cv2.imshow("YOLOv10 Detection on Image", frame)
        cv2.waitKey(0)  # Nhấn phím bất kỳ để đóng cửa sổ hiển thị

else:
    #### Mở video và xử lý từng frame nếu là video
    cap = cv2.VideoCapture(video_path)

    while cap.isOpened():
        success, frame = cap.read()

        if success:
            # Thực hiện inference YOLOv8
            results = model(frame)

            # Kiểm tra nếu có bất kỳ đối tượng nào được phát hiện
            if len(results[0].boxes) > 0:
                for box in results[0].boxes:
                    confidence = box.conf[0]
                    label = box.cls
                    label_name = model.names[int(label)]

                    # Kiểm tra nếu độ tin cậy trên ngưỡng
                    if confidence > confidence_threshold:
                        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                        detected_object = frame[y1:y2, x1:x2]
                        frame_filename = os.path.join(output_folder, f"{label_name}_{confidence:.2f}_frame_{frame_count:04d}.jpg")
                        cv2.imwrite(frame_filename, detected_object)
                        color = colors.get(label_name, (255, 255, 255))
                        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                        text = f"{label_name} {confidence:.2f}"
                        cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

                cv2.imshow("YOLOv10 Inference on Video", frame)
                frame_count += 1

                if cv2.waitKey(1) & 0xFF == ord("q"):
                    break
        else:
            break

    cap.release()

cv2.destroyAllWindows()

FileNotFoundError: [Errno 2] No such file or directory: '../image/xemay/thanh-nien-dung-tren-yen-xe-may-dang-chay.jpg'

In [ ]:
import cv2
print(cv2.getBuildInformation())

In [4]:
# Cắt tất cả thành hình ảnh
import cv2
import os

# Đường dẫn tới video
video_path = './predict_data/bike_counter_10min.mp4'  # Thay đổi đường dẫn tới video của bạn
video_name = os.path.splitext(os.path.basename(video_path))[0]

# Tạo thư mục lưu frame nếu chưa có
output_folder = 'detected_frames'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Đọc video
cap = cv2.VideoCapture(video_path)

frame_count = 0
saved_frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Lấy mỗi 3 frame
    if frame_count % 3 == 0:
        frame_filename = f"{video_name}_frame_{saved_frame_count}.jpg"
        frame_path = os.path.join(output_folder, frame_filename)
        
        # Lưu frame
        cv2.imwrite(frame_path, frame)
        saved_frame_count += 1

    frame_count += 1

# Giải phóng video
cap.release()
cv2.destroyAllWindows()

print(f"Lưu {saved_frame_count} frame vào thư mục '{output_folder}'")

Lưu 5001 frame vào thư mục 'detected_frames'


In [21]:
#!pip install roboflow

# from roboflow import Roboflow
# rf = Roboflow(api_key="HTzRcoTwwe0y2jQf0bvN")
# project = rf.workspace("labelimgbiensoxe").project("biensoxe_all-zrwrj")
# version = project.version(8)
# dataset = version.download("yolov8")

from roboflow import Roboflow
rf = Roboflow(api_key="8joVH7TL2k5V6uScFBEy")
project = rf.workspace("moin").project("car_license_plates")
version = project.version(2)
dataset = version.download("yolov8")            

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.100, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Car_License_Plates-2 in yolov8:: 100%|██████████| 1436/1436 [00:00<00:00, 2582.75it/s]


In [16]:
import cv2
import os
from ultralytics import YOLO

# Load model YOLO
#model = YOLO("./Models_yolov10n_dataBienSoNhieuLoaiv1/runs/detect/train/weights/last.pt")  
model = YOLO("./Models_yolov10n_dataBienSoNhieuLoaiv2/runs/detect/train/weights/last.pt")

# Đường dẫn đầu vào cho ảnh hoặc video
input_path = "./predict_data/anh_anhsuutam_24.jpg"  # Thay bằng đường dẫn tới ảnh hoặc video của bạn

output_folder = "./detected_frames"
os.makedirs(output_folder, exist_ok=True)  # Tạo thư mục nếu chưa tồn tại

# Kiểm tra xem input là ảnh hay video
is_image = input_path.lower().endswith(('.jpg', '.jpeg', '.png'))

# Ngưỡng độ tin cậy cho việc hiển thị và lưu đối tượng
confidence_threshold = 0.5  # Thay đổi theo nhu cầu

if is_image:
    # Nếu là ảnh, đọc và xử lý ảnh
    image = cv2.imread(input_path)
    results = model(image)

    # Xử lý từng đối tượng được phát hiện
    for box in results[0].boxes:
        confidence = box.conf[0]
        if confidence >= confidence_threshold:
            label = box.cls
            label_name = model.names[int(label)]
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())

            # Cắt và lưu đối tượng được phát hiện
            detected_object = image[y1:y2, x1:x2]
            output_filename = f"{label_name}_{confidence:.2f}.jpg"
            output_path = os.path.join(output_folder, output_filename)
            cv2.imwrite(output_path, detected_object)

            # Vẽ bounding box và nhãn lên ảnh gốc
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(image, f"{label_name} {confidence:.2f}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    # Hiển thị ảnh đã annotate
    cv2.imshow("Detected Objects", image)
    cv2.waitKey(0)

else:
    # Nếu là video, đọc và xử lý từng frame
    cap = cv2.VideoCapture(input_path)
    frame_count = 0

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        results = model(frame)
        for box in results[0].boxes:
            confidence = box.conf[0]
            if confidence >= confidence_threshold:
                label = box.cls
                label_name = model.names[int(label)]
                x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())

                # Cắt và lưu đối tượng được phát hiện
                detected_object = frame[y1:y2, x1:x2]
                output_filename = f"{label_name}_{confidence:.2f}_frame_{frame_count:04d}.jpg"
                output_path = os.path.join(output_folder, output_filename)
                cv2.imwrite(output_path, detected_object)

                # Vẽ bounding box và nhãn lên frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f"{label_name} {confidence:.2f}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        # Hiển thị frame đã annotate
        cv2.imshow("Detected Objects", frame)
        frame_count += 1

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()

# Đóng cửa sổ hiển thị
cv2.destroyAllWindows()


0: 448x640 1 bien_so_quansu, 1 bien_so_xanh, 54.6ms
Speed: 5.0ms preprocess, 54.6ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)
